In [6]:
import requests
import trafilatura
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import modal

In [7]:
print(torch.__version__)
print("MPS available:", torch.backends.mps.is_available())

2.6.0.dev20241112
MPS available: True


In [8]:
if torch.backends.mps.is_available():
    print("🔥 MPS is available!")
    print(f"Using device: {torch.device('mps')}")
else:
    print("❌ MPS not available, check installation!")


🔥 MPS is available!
Using device: mps


In [9]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

finbert_model = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(finbert_model)
model = AutoModelForSequenceClassification.from_pretrained(finbert_model).to(device)

sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if device.type == "mps" else -1)

print("FinBERT loaded on:", device)

FinBERT loaded on: mps


In [11]:
stock_articles = [
    "Apple's earnings report exceeded expectations.",
    "Tesla stock saw a significant drop in the last quarter.",
    "Amazon is expanding its delivery network in Europe."
]
results = sentiment_pipeline(stock_articles)
for res in results:
    print(res)

{'label': 'positive', 'score': 0.9441174268722534}
{'label': 'negative', 'score': 0.9740584492683411}
{'label': 'positive', 'score': 0.9288909435272217}


In [16]:
result = sentiment_pipeline("I believe that stocks are exceeding expectations")
print(result)

[{'label': 'positive', 'score': 0.9290725588798523}]


In [24]:
import openai
from fastapi import FastAPI, Query
 
app = FastAPI()

# Load FinBERT model for sentiment analysis
finbert_model = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(finbert_model)
model = AutoModelForSequenceClassification.from_pretrained(finbert_model)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if device.type == "mps" else -1)

# OpenAI API key (ensure you set this securely in a real application)
openai.api_key = "your-openai-api-key"

@app.get("/analyze")
def analyze_stock(ticker: str = Query(..., description="Stock ticker symbol")):
    """
    Analyze the sentiment of a given stock ticker and provide investment reasoning.
    """
    # Placeholder for news/article retrieval (to be implemented later)
    news_article = f"Latest news about {ticker}. Apple's earnings report exceeded expectations."
    
    # Perform sentiment analysis
    sentiment_result = sentiment_pipeline(news_article)[0]
    sentiment_label = sentiment_result["label"]
    
    # Generate investment reasoning using OpenAI
    reasoning_prompt = (
        f"The sentiment for stock {ticker} is {sentiment_label}. Explain why this sentiment might impact investment decisions."
    )
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": reasoning_prompt}]
    )
    investment_reasoning = response["choices"][0]["message"]["content"]
    
    return {
        "ticker": ticker,
        "sentiment": sentiment_label,
        "reasoning": investment_reasoning
    }

In [ ]:
AAPL